## PCX - Phuture Crypto Market Index - Methodology Template

## Configuring packages

In [5]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [6]:
min_mcap =  1e8
min_weight = 0.005
max_weight = 0.2
circ_supply_threshold = 0.3
liveness_threshold = 365 # In days
liquidity_consistency = 90 # In days
coingecko_category = []

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')
liquid_staking_tokens = pd.DataFrame(cg.get_coins_markets('usd',category='liquid-staking-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                100000,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [7]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens,liquid_staking_tokens],
    ids_to_replace=[['bitcoin'],['wrapped-bitcoin']],
    remove_category_assets=['wrapped-bitcoin'],
    values_to_update=[{'id':'matic-network','category':'platforms','value':{}}],
    platforms_to_remove=['ethereum']

    )
liquidity

Excluding mantle, pricing data available only for 284 < 365 days
Excluding wrapped-eeth, pricing data available only for 114 < 365 days
Excluding beam-2, pricing data available only for 183 < 365 days
Excluding safe, pricing data available only for 3 < 365 days
Excluding aerodrome-finance, pricing data available only for 223 < 365 days
Excluding corgiai, pricing data available only for 290 < 365 days
Excluding based-brett, pricing data available only for 57 < 365 days
Excluding degen-base, pricing data available only for 102 < 365 days
Excluding mog-coin, pricing data available only for 280 < 365 days
polygon-zkevm not supported
sora not supported
cronos not supported
zksync not supported
energi not supported
tomochain not supported
harmony-shard-0 not supported
moonriver not supported
boba not supported
metis-andromeda not supported
xdai not supported
aurora not supported
fuse not supported
meter not supported
moonbeam not supported
telos not supported
syscoin not supported
milkomeda-

/Users/olivermehr/Documents/GitHub/rebalancing-data/PCX/../methodology.py:444: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  slippages["best slippage chain"] = slippages.idxmax(


Dropping echelon-prime because a price feed is unavailable
gmx does not meet the minimum weight requirement


,arbitrum-one,avalanche,base,binance-smart-chain,fantom,optimistic-ethereum,polygon-pos,optimal chain,best slippage,best slippage chain
id,,,,,,,,,,
binancecoin,NaN,NaN,NaN,-0.001176,NaN,NaN,NaN,binance-smart-chain,-0.001176,binance-smart-chain
wrapped-bitcoin,-0.001812,-0.007429,NaN,NaN,-1.000000,-1.000000,-0.003153,arbitrum-one,-0.001812,arbitrum-one
avalanche-2,NaN,-0.002723,NaN,NaN,NaN,NaN,NaN,avalanche,-0.002723,avalanche
matic-network,NaN,NaN,NaN,NaN,NaN,NaN,-0.003609,polygon-pos,-0.003609,polygon-pos
chainlink,-0.005312,-0.242362,NaN,-0.106174,-0.225322,-0.132325,-0.009958,arbitrum-one,-0.005312,arbitrum-one
...,...,...,...,...,...,...,...,...,...,...
rollbit-coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
threshold-network-token,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
ssv-network,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


## Show Results

In [8]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
wrapped-bitcoin,BTC,1251679418887,0.200677,13152,0x2f2a2543b76a4166549f7aab2e75bef0aefc5b0f,arbitrum-one
binancecoin,BNB,93428728398,0.200677,13152,0x0000000000000000000000000000000000000000,binance-smart-chain
avalanche-2,AVAX,13363453441,0.175550,11505,0x0000000000000000000000000000000000000000,avalanche
chainlink,LINK,8552846216,0.112355,7364,0xf97f4df75117a78c1a5a0dbb814af92458539fb4,arbitrum-one
matic-network,MATIC,6601997183,0.086728,5684,0x0000000000000000000000000000000000000000,polygon-pos
uniswap,UNI,5900485951,0.077512,5080,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,arbitrum-one
render-token,RNDR,3262933082,0.042864,2809,0x61299774020da444af134c82fa83e3810b309991,polygon-pos
the-graph,GRT,2531386986,0.033254,2179,0x9623063377ad1b27544c965ccd7342f7ea7e88c7,arbitrum-one
lido-dao,LDO,1788660164,0.023497,1539,0xc3c7d422809852031b44ab29eec9f1eff2a58756,polygon-pos
